# Session 15: Visualization

In [11]:
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

from ipyleaflet import Map, LocalTileLayer, WMSLayer, projections

from grouputils import initialize_rasterizer

Interactive plotting is an excellent tool to facilitate data exploration. A very popular tool for creating Interactive maps is [Leaflet](https://leafletjs.com/). Leaflet is a JavaScript library for mapping, which has wrappers in numerous languages including python (`ipyleaflet`).

In the previous two sessions, we created tiles at the maximum zoom level (z = 13). If you wanted to display all of this data on an interactive map, like a Leaflet map, it would be extremely slow to load because there is so much of it, and it's at such a high resolution. To show this data in a performant way on a map, we need to create _lower_ resolution tiles ("downsampling"), and they need to be in a web-accessible format (most browsers can't render GeoTIFF files, so PNG is better.) - [Here is a good explanation](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/)

Today, we will create web tiles as PNG images of those tiles at all zoom levels, from 1 to 13. This way, when we are zoomed _out_ in the interactive map, the _lower_ resolution zoom level 1 is loaded. Higher resolution levels load only when you zoom _in_, and the extent you are trying to view is smaller, so fewer tiles load. This scaling of resolution by zoom level is what allows us to interact with the data in a performant way.

First, we'll set up the rasterizer as before.

In [12]:
iwp_rasterizer = initialize_rasterizer("/home/shares/example-pdg-data")

Here are the methods that we will use from the RasterTiler:

1. `parent_geotiffs_from_children`, which takes as an argument the set of tiles (GeoTIFFs) that we produced yesterday, and produces parent tiles at the next zoom level.

2. `webtiles_from_geotiffs` takes the set of GeoTIFFs produced by `parent_geotiffs_from_children` and creates PNG web tiles from them.
    - Note: PNG stands for "Portable Network Graphic", which is a type of raster image file!

Like the last session, we will run this code in batches.

In [13]:
# We'll also use the make_batch function to create batches of
# GeoTIFF files to process.
def make_batch(items, batch_size):
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

## Creating parent tiles

To create lower resolution GeoTIFF files, we can combine high resolution GeoTIFFs then resample them so that we still have 256x256 pixel data. We will start with zoom level 12, and run the `parent_geotiffs_from_children` method on each zoom level to generate the parent zoom level. Then we run it on the parent level to generate the grandparent level...and so on.

In [14]:
# Get each z-level of GeoTIFFs we need to create:
parent_zs = list(range(12, 0, -1))
print(f'Parent Zs: {parent_zs}')

Parent Zs: [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


Here is an example for how we can run this, without `parsl`, for just 1 z-level.

In [15]:
parent_z = 12
batch_size = 50

# get a list of child paths from the parent zoom level
all_child_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff', z = parent_z + 1) # z-level 13

# Make a list of all the parent tiles that will be created from the z+1 child
# tiles
all_parent_tiles = set()
for child_path in all_child_paths:
    parent_tile = iwp_rasterizer.tiles.get_parent_tile(child_path)
    all_parent_tiles.add(parent_tile)

all_parent_tiles = list(all_parent_tiles)

# Break all parent tiles at level z into batches
parent_tile_batches = make_batch(all_parent_tiles, batch_size)

# write the geotiff files for z-level 12
for parent_tile_batch in parent_tile_batches:
    # crate rasters for z-level 12 from aggregated z-level 13 rasters
    iwp_rasterizer.parent_geotiffs_from_children(parent_tile_batch, recursive = False)


Check out the z-levels within the `geotiff` directory. There is now a z-level 12 that is populated with files.

Now that we know how to do this for one zoom level...

- How would you write this out for all of the levels?
- In what part of the code would you want to insert a `@python_app` decorator to parallelize the process?
- What dependencies exist for this parallel process to run correctly?

Write a parsl app to parallelize generating the parent tiles.

In [16]:
# Generate parsl app for creating parent geotiffs from children

@python_app
def downsample_parent_tile_batch(parent_tile_batch, iwp_rasterizer):
    # crate rasters for z-level z from aggregated z-level z+1 rasters
    iwp_rasterizer.parent_geotiffs_from_children(parent_tile_batch, recursive = False)
    return parent_tile_batch

def downsample(parent_z, iwp_rasterizer, batch_size = 10):
    print(f'Downsampling to parent z-level {parent_z}')
    # get a list of child paths from the parent zoom level
    all_child_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff', z = parent_z + 1) # z-level z+1

    # Make a list of all the parent tiles that will be created from the z+1 child
    # tiles
    all_parent_tiles = set()
    for child_path in all_child_paths:
        parent_tile = iwp_rasterizer.tiles.get_parent_tile(child_path)
        all_parent_tiles.add(parent_tile)

    all_parent_tiles = list(all_parent_tiles)

    # Break all parent tiles at level z into batches
    parent_tile_batches = make_batch(all_parent_tiles, batch_size)

    futures = []
    for batch in parent_tile_batches:
        # create rasters for z-level z from aggregated z-level z+1 rasters
        future = downsample_parent_tile_batch(batch, iwp_rasterizer)
        futures.append(future)
        print(f'Assembled future for: {batch_size}')
    
    for future in futures:
        downsampled_batch = future.result()
        print(f'Downsampled parent tile batch: {downsampled_batch}')

Next, set up your executor as we did yesterday.

In [17]:
# Set up executor
activate_env = 'workon scomp'
htex_local = Config(
    executors = [
        HighThroughputExecutor(
            max_workers_per_node = 11, # Caps the number of workers launched per node
            provider = LocalProvider(
                worker_init = activate_env,
                max_blocks = 1
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

Then generate parent tiles at all zoom levels.

In [18]:
# generate parent tiles at all zoom levels using the parsl app
for parent_z in parent_zs:
    downsample(parent_z, iwp_rasterizer, batch_size = 10)

Downsampling to parent z-level 12
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Assembled future for: 10
Downsampled parent tile batch: [Tile(x=456, y=539, z=12), Tile(x=459, y=536, z=12), Tile(x=450, y=533, z=12), Tile(x=459, y=540, z=12), Tile(x=446, y=538, z=12), Tile(x=458, y=529, z=12), Tile(x=462, y=533, z=12), Tile(x=459, y=538, z=12), Tile(x=449, y=538, z=12), Tile(x=453, y=530, z=12)]
Downsampled parent tile batch: [Tile(x=448, y=540, z=12), Tile(x=448, y=529, z=12), Tile(x=463, y=532, z=12), Tile(x=457, y=536, z=12), Tile(x=458, y=531, z=12), Tile(x=462, y=539, z=12), Tile(x=456, y=529, z=12), Tile(x=448, y=533, z=12), Tile(x=460, y=534, z=12), Tile(x=447, y=539, z=12)]
Downsam

Great! Now all zoom levels have been created and populated with `.tif` files in the `geotiff` directory.

# Make web tiles

In the last step to create the web tiles for the map, we will use the `webtiles_from_geotiffs` method on all of the GeoTIFFs we just created. First, we need to update our `iwp_rasterizer` config object using the `update_ranges` method, which will add the new zoom ranges we just created to the config information.

In [19]:
iwp_rasterizer.update_ranges()

Now we can get a list of files to create webtiles from, and run the `webtile_from_geotiff` (not plural) function on just one geotiff file.

In [20]:
geotiff_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff')
iwp_rasterizer.webtile_from_geotiff(geotiff_paths[0])

Tile(x=909, y=1059, z=13)

Notice that there is a new directory called web_tiles, which contains 2 PNG image of the 1 GeoTIFF file input. Each PNG image represents a statistic calculated for that GeoTIFF: number of ice wedge polygons per pixel, and proportion of each pixel that is covered by ice wedge polygons.

Saving the rasters as PNG's is a form of "lossless" compression. This means that data is not actually lost when we convert it, and it can be reconstructed. An example of "lossy" compression would be saving the data as JPEG's.

Like we've done before, create a `parsl` app to create the web tiles in parallel. For `webtiles_from_geotiffs` (plural), set `update_ranges = False` because already did that above.

In [21]:
# create parsl app

@python_app
def create_webtiles(geotiff_batch, iwp_rasterizer):
    iwp_rasterizer.webtiles_from_geotiffs(geotiff_batch, update_ranges = False)
    return geotiff_batch

In [22]:
# run app over all of the geotiffs in batches of 200

geotiff_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff')
geotiff_batches = make_batch(geotiff_paths, batch_size = 200)

futures = []
for batch in geotiff_batches:
    future = create_webtiles(batch, iwp_rasterizer)
    futures.append(future)
    print(f'Assembled future for {batch}')
    
for future in futures:
    maptile_batch = future.result()
    print(f'Created webtiles for {future.result()}')

Assembled future for ['geotiff/WGS1984Quad/13/909/1059.tif', 'geotiff/WGS1984Quad/13/909/1060.tif', 'geotiff/WGS1984Quad/13/909/1061.tif', 'geotiff/WGS1984Quad/13/909/1068.tif', 'geotiff/WGS1984Quad/13/909/1069.tif', 'geotiff/WGS1984Quad/13/909/1067.tif', 'geotiff/WGS1984Quad/13/909/1062.tif', 'geotiff/WGS1984Quad/13/909/1063.tif', 'geotiff/WGS1984Quad/13/910/1079.tif', 'geotiff/WGS1984Quad/13/910/1080.tif', 'geotiff/WGS1984Quad/13/910/1081.tif', 'geotiff/WGS1984Quad/13/910/1061.tif', 'geotiff/WGS1984Quad/13/910/1082.tif', 'geotiff/WGS1984Quad/13/910/1062.tif', 'geotiff/WGS1984Quad/13/910/1083.tif', 'geotiff/WGS1984Quad/13/910/1067.tif', 'geotiff/WGS1984Quad/13/910/1063.tif', 'geotiff/WGS1984Quad/13/910/1068.tif', 'geotiff/WGS1984Quad/13/910/1069.tif', 'geotiff/WGS1984Quad/13/910/1070.tif', 'geotiff/WGS1984Quad/13/910/1071.tif', 'geotiff/WGS1984Quad/13/910/1072.tif', 'geotiff/WGS1984Quad/13/910/1073.tif', 'geotiff/WGS1984Quad/13/910/1074.tif', 'geotiff/WGS1984Quad/13/910/1059.tif', 'ge

In [23]:
# Shutdown and clear the parsl executor
htex_local.executors[0].shutdown()
parsl.clear()

Wonderful! Now we have created web tiles for all zoom levels. Don't forget to shut down the executor.

## Make a map!

Now we are finally ready to explore these data using `ipyleaflet`. First, we have to set up a basemap layer to use. We use a set of WMS (Web Map Service) tiles provided by USGS. One reason we chose this set is it allows us to reproject the data into EPSG 4326, which is the projection our data are in. Other commonly used web tiles are in web mercator, which is not what our data are in.

In [24]:
wms = WMSLayer(
    url="https://basemap.nationalmap.gov:443/arcgis/services/USGSImageryTopo/MapServer/WmsServer",
    layers="0",
    format="image/png",
    transparent=True,
    min_zoom=0,
    crs=projections.EPSG4326
)

Now we can call the `Map` function from `ipyleaflet`, set up a default zoom and center, add the WMS layer and projection, and finally add our set of local tiles using the `add_layer` method. 

Note that the path you give to the `LocalTileLayer` function has variables for z, x, and y. If you look in the `geotiff` directory at the higher zoom levels, you'll see that the directories and files are all named in a very standardized fashion. This is so that they can be automatically handled by the `LocalTileLayer` function. This is a standard for how web tiles are served. The methods from the `rasterizer` class handled this for us in our data processing.

In [25]:
from ipyleaflet import Map, LocalTileLayer, WMSLayer, projections, WidgetControl
from ipywidgets import HTML

map = Map(center=(66.5, -159.9), zoom=9, layers=(wms,), crs=projections.EPSG4326)

# Add indicator to show what level of zoom
zoom_label = HTML(value=f'Zoom level: {map.zoom}')
widget_control = WidgetControl(widget=zoom_label, position='topright')
map.add_control(widget_control)

# Update zoom label when map zoom changes
def update_zoom(change):
        zoom_label.value = f'Zoom level: {change.new}'

map.observe(update_zoom, names='zoom')

map.add_layer(LocalTileLayer(path='web_tiles/prop_pixel_covered_by_IWP/WGS1984Quad/{z}/{x}/{y}.png'));

map

Map(center=[66.5, -159.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [26]:
# Get current zoom level
map.zoom

9.0

## Bonus

Play around with [`ipyleaflet`](https://ipyleaflet.readthedocs.io/en/latest/) and add whatever features you think might be useful, like a legend or markers.